In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-publisher-dna-log.v1/fact/granularity=daily/date=2020-08-10/



In [0]:
%%sh
# aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.app-tag-mapping.v1/dimension/_update_date=2020-08-06/ --recursive
# echo "******"
# aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.app-tag-mapping.v1/dimension/_update_date=2020-08-11/ --recursive
# echo "******"
aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.company-hq-mapping.v1/dimension/_update_date=2020-10-02/  --recursive
aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.company-hq-mapping.v1/dimension/_update_date=2020-10-04/  --recursive

# echo "******"
# aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.company-hq-mapping.v1/dimension/_update_date=2020-08-11/  --recursive
# echo "******"
# aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.in-unified-product-product-mapping.v1/dimension/_update_date=2020-08-06/  --recursive
# echo "******"


# aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/update_date=2020-08-03/


# aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.in-unified-product-product-mapping.v1/dimension/_update_date=2020-08-13/  --recursive



In [0]:



# print spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.app-tag-mapping.v1/dimension/").where("_update_date='2020-07-27' and _update_hour=1").where("tag_type_code='modifier'").count()



spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.app-tag-mapping.v1/dimension/").where("_update_date='2020-08-09' and _update_hour=7").where("tag_type_code='modifier'").createOrReplaceTempView("temp")
spark.sql("select * from temp where tag_type_code = 'modifier' and app_id = 20600009921109 and tag_id=600046 and parent_tag_id=400002").show()


In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.in-unified-product-product-mapping.v1/dimension/_update_date=2020-08-11/_update_hour=8/part-00000-7036e060-5c60-411b-a317-b6a4f7f0ec84.c000.gz.parquet").show()

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
-- set search_path=dna;
--\d
-- SELECT column_name, data_type FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'dna_app_unified_dim_v2';
-- select count(*) from dna_app_tag_dim_v1 where tag_type_code = 'modifier' limit 2;

--select * from dna.dna_app_tag_dim_v1 limit 3;
--select * from dna.dna_company_hq_dim_v1 limit 3;

--select count(*) from dna_app_tag_dim_v1 where tag_type_code = 'genre'  limit 5;

-- select count(*) from dna.dna_app_unified_dim_v2 limit 3;
select * from productized_index_config limit 1;

EOF


In [0]:

print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.company-hq-mapping.v1/dimension/_update_date=2020-07-17/").show()


In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
def citus_row():
    def get_data_in_citus():
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select app_id,tag_id,parent_tag_id from dna.dna_app_tag_dim_v1 where tag_type_code='genre'"
        db_data = query(citus_dsn_, sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus()
    return [Row(app_id=r[0], tag_id=r[1], parent_tag_id=r[2] ) for r in result]
df = citus_row()

    # return [Row(device_code=r[0], sum_est_free_app_download=r[1], sum_est_paid_app_download=r[2],
    #             sum_est_revenue=r[3]) for r in result]


In [0]:

# print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.in-unified-product-product-mapping.v1/dimension/_update_date=2020-07-14/").show()

df_unified = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.app-tag-mapping.v1/dimension/").where("_update_date='2020-08-09' and _update_hour=7").where("tag_type_code='genre'").select("app_id","tag_id","parent_tag_id")
df_db = spark.createDataFrame(df)

df_unified.createOrReplaceTempView("unified")
df_db.createOrReplaceTempView("db")


# spark.sql("select count(1) from db").show()
# spark.sql("select count(1) from unified").show()

spark.sql("select app_id, tag_id,  parent_tag_id from db except all select app_id, tag_id,  parent_tag_id from unified").show(200)
spark.sql("select app_id, tag_id,  parent_tag_id from unified except all select app_id, tag_id,  parent_tag_id from db").show(200)



In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.in-unified-product-product-mapping.v1/dimension/_update_date=2020-07-11/").where("app_id=20600012508105").show()

In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.in-unified-product-product-mapping.v1/dimension/").where("app_id=20600012508105").show()

In [0]:


import datetime
import traceback
from functools import wraps
from unittest import skipIf
import psycopg2
from conf import settings
import croniter
from aadatapipelinecore.core.utils import email
from aadatapipelinecore.core.log import logger
from pandas.io import sql as sqlio

def _get_date_from_refresh_routing_config(*config):
    """
    return the date of : <days_delta> ago from previous scheduled date&time according to <cron_time>.
    e.g.
    config = ("0 9 * * *", 1), today is 2019-10-27 8:00
    so previous scheduled date&time is 2019-10-26 9:00
    will return "2019-10-25"

    e.g.
    config = ("0 7 * * *", 1), today is 2019-10-27 8:00
    so previous scheduled date&time is 2019-10-27 7:00
    will return "2019-10-26"

    :param config: config format: (<cron_time>, <days_delta>)
    :type config: tuple
    the cron_time please refer to https://support.acquia.com/hc/en-us/articles/360004224494-Cron-time-string-format
    the days_delta is the days ago from the expected date.
    :return: date string of "%Y-%m-%d"
    :type return: str
    """
    if len(config) == 1:
        schedule, days_delta = config
        # here use UTC now
        cron = croniter.croniter(schedule, datetime.datetime.utcnow())
        date = cron.get_prev(datetime.datetime) - datetime.timedelta(days=days_delta)
        print date
        return date.strftime("%Y-%m-%d")
    elif len(config) == 2:
        schedule, days_delta = config[0]
        # here use UTC now
        cron = croniter.croniter(schedule, datetime.datetime.utcnow())
        print 'previously', cron.get_prev(datetime.datetime)
        print 'delta', datetime.timedelta(days=days_delta)
        date = cron.get_prev(datetime.datetime) - datetime.timedelta(days=days_delta)
        # print date
        return date.strftime("%Y-%m-%d"), date.strftime("%-H")

trigger_date_config = ("0 1,7,13,19 * * *", 0)

date , hour = _get_date_from_refresh_routing_config(trigger_date_config, "hourly")
print date, hour

In [0]:

from applications.db_check_v1.common.constants import query, citus_settings
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

def get_dna_db_data(sql):
    result = query(citus_settings("store"), sql)
    return result


In [0]:


# print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.app-tag-mapping.v1/dimension/_update_date=2020-07-11/_update_hour=7/").where("tag_type_code = 'genre'").count()
print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.company-hq-mapping.v1/dimension/_update_date=2020-07-11/_update_hour=2/").show()
print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.company-hq-mapping.v1/dimension/_update_date=2020-07-13/_update_hour=2/").count()


In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=dna;
select tag_type_code, count(*) from dna.dna_app_tag_dim_v1 where tag_type_code in ('genre', 'modifier') group by tag_type_code   limit 3;
select count(*) from dna.dna_company_hq_dim_v1 limit 3;

select tag_type_code, count(1) from dna.dna_app_tag_dim_v1 where tag_type_code in ('genre', 'modifier') group by tag_type_code;
select * from dna.dna_app_unified_dim_v1 limit 3;
EOF

In [0]:


spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.company-hq-mapping.v1/dimension/_update_date=2020-07-07/_update_hour=14/").show()


In [0]:
%%sh


PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;

-- universal_id : company id
-- parent_id : parent id
--select * from dna_universal_company_parent where universal_id = 1000200000229763 limit 5;
-- select * from dna_universal_company_parent where universal_id  = 1000200000000088 limit 5;
-- select * from ( select universal_id, count(universal_id) from dna_universal_company_parent where start_date is not null and end_date is not null group by universal_id ) as prod limit 5

-- select * from dna_universal_company_parent where start_date < '2012-01-01' limit 10;


-- universal_id : company id
-- company id : publisher id
-- select * from dna_universal_company_mapping where company_id=542280115   limit 2;

--select * from dna_universal_company_parent where universal_id=1000200000229763   limit 2;
select company_id from dna_universal_company_mapping where universal_id = 1000200000035578 ;

select * from 
EOF

In [0]:
%%sh
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/market_code=apple-store/batch_id=1/part-applestore-1.c000.gz.parquet

# aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2018-12-16/platform=android/
 
# aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/
aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2014-11-24/ios/sbe_est_app/143441/

In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2  -U app_tomcat -d aa_android -p 5433 << EOF
BEGIN;
SELECT 
    *
    from(
        select 
        distinct app_id, type, store_id, old_value, new_value, date
        FROM event 
        where type=33 and date between '2020-01-01' and '2020-01-03' limit 3
        ) as prod

LIMIT 10;
rollback;
EOF

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=dna;
-- select * from dna.dna_mapping_log_4app_dim_v1 where end_date is null limit 3;
-- select distinct publisher_id , company_id, parent_company_id, start_date, end_date  from dna.dna_mapping_log_4pub_dim_v1 where company_id =1000200000019824 limit 6;



--select count(1) from dna.dna_app_tag_dim_v1 where tag_type_code != 'custom'   limit 3;
--select * from dna.dna_app_tag_dim_v1 where tag_type_code != 'custom'  order by updated_time desc limit 3;

--select count(*) from dna.dna_app_unified_dim_v1 limit  3;
-- select count(*) from dna.dna_company_hq_dim_v1  limit 3;
select * from dna.dna_mapping_log_4app_dim_v1 limit 3;

EOF



In [0]:
%%sh

#  universal_id   |    parent_id     | start_date | end_date
# 1 ｜ 2 ｜ 2020-01-01 | 2020-02-01
# 1 ｜ 3 ｜  2020-02-02 | 
# 1 ｜ 4 ｜  2020-01-01 |  2020-01-02 | 

PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;

-- universal_id : company id
-- parent_id : parent id
-- select * from dna_universal_company_parent where universal_id = 1000200000003635 limit 5
-- select * from dna_universal_company_parent where universal_id  = 1000200000000088 limit 5;
-- select * from ( select universal_id, count(universal_id) from dna_universal_company_parent where start_date is not null and end_date is not null group by universal_id ) as prod limit 5

-- select * from dna_universal_company_parent where start_date < '2012-01-01' limit 10;


-- universal_id : company id
-- company id : publisher id
-- select * from dna_universal_company_mapping where company_id=542280115   limit 2;

select * from dna_universal_company_parent where universal_id=1000200000019824   limit 2;


EOF

In [0]:


spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-03-31").createOrReplaceTempView("publisher_test")
spark.sql("select * from publisher_test where platform='ios' and category_id = 36 and feed=0 order by rank desc ").show()

In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2  -U app_tomcat -d aa -p 5432 << EOF
BEGIN;
SELECT 
    *
FROM aa_event
WHERE  type=33 
ORDER BY date DESC
LIMIT 5 
;
EOF


In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2  -U app_tomcat -d aa_android -p 5433 << EOF
BEGIN;
SELECT 
    *
    from(
        select 
        distinct app_id, type, store_id, old_value, new_value 
        FROM event 
        where type=14 and date between '2020-01-01' and '2020-02-01'
        ) as prod

LIMIT 10;
rollback;
EOF

In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2  -U app_tomcat -d aa_android -p 5433 << EOF
BEGIN;
\d event
select  app_id, store_id, type, old_value, new_value, date
FROM event 
where type=14
and date between '2020-01-01' and '2020-02-01'

LIMIT 10;
rollback;
EOF

In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
SELECT  count(distinct pub_id) as count_pub_id
 FROM plproxy.execute_select_nestloop(\$proxy\$ 
  SELECT distinct pub_id 
    FROM pp.pub_store_daily_estimate_0
    WHERE date='2020-01-01'
    LIMIT 10
     \$proxy\$) tbl (pub_id BIGINT)  LIMIT 10 ;

EOF
 
 



In [0]:
%%sh


In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
SELECT  pub_id, category_id, app_id, date 
 FROM plproxy.execute_select_nestloop(\$proxy\$ 
  SELECT pub_id, category_id, app_id, date
    FROM pp.pub_app_daily_associate_0 order by date asc
    LIMIT 10
     \$proxy\$) tbl (pub_id BIGINT, category_id INTEGER, app_id BIGINT, date DATE )  LIMIT 10 ;

EOF

In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
SELECT distinct company_id, pub_id, company_type, date_range, date 
 FROM plproxy.execute_select_nestloop(\$proxy\$ 
  SELECT company_id, pub_id, company_type, date_range, date
    FROM pp.company_publisher_map_0
    where pub_id=20200000762856
    order by pub_id
    LIMIT 1000;
     \$proxy\$) tbl (company_id BIGINT, pub_id BIGINT, company_type SMALLINT, date_range DATERANGE, date DATE)  LIMIT 1000 ;

EOF
 
 

# CompanyType = Enum({
#     "PARENT_COMPANY": (1, "Parent company"),
#     "SUBSIDIARY": (2, "Subsidiary"),
#     "COMPANY_WITHOUT_PARENT": (3, "company without parent and not parent company")
# })



In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;
-- universal_id : company id
-- parent_id : parent id
-- select * from dna_universal_company_parent where parent_id in (1000200000000033) and disabled='f' and confirmed='t' limit 1000;
-- select * from dna_universal_company_parent where start_date < '2012-01-01' limit 10;



-- universal_id : company id
-- company id : publisher id
-- select * from dna_universal_company_mapping where company_id=542280115   limit 100;


select * from dna_genre_id_product_mapping order by last_updated_time desc limit 5;

-- select * from in_unified_product_product_mapping  limit 3;
-- select * from in_app_dna_info_mapping limit 3;

-- select * from dna_genre_mapping where parent_genre_id is null limit 5;
-- select * from dna_universal_company_office limit 3

-- \d dna_universal_company_office
-- select * from dna_universal_company_office where type != 'HQ' limit 3

EOF
{1,102,1000}


In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;

-- select * from dna_universal_company_parent where universal_id=1000200000003635 limit 3;
-- select * from dna_universal_company_mapping where universal_id=1000200000003635 limit 3;
-- select * from dna_deleted_universal_company_mapping where company_name is not null limit 3;
--select * from dna_deleted_universal_company_mapping where universal_company_id=1000200000020314  limit 5;




SELECT 
         s.parent_id AS company_id,
         t.company_id AS pub_id,
         1 AS company_type,
     daterange(coalesce(s.start_date, '-infinity'), coalesce(s.end_date,'infinity'), '[)') AS date_range,
         now()::date AS date
   FROM dna_universal_company_parent s, dna_universal_company_mapping t
   WHERE 
         s.universal_id = t.universal_id AND
         NOT s.disabled AND
         s.confirmed AND
         NOT t.disabled AND
         t.confirmed AND
         t.market = 'gp'
    GROUP BY s.parent_id, t.company_id, date_range 
    order by company_id desc



EOF

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;

\d
-- 284882218
-- SELECT * FROM dna_universal_company AS a JOIN dna_universal_company_office AS b ON a.id=b.universal_company_id WHERE a.disabled=false AND b.type!='HQ' and b.country IS NOT NULL limit 3;
--select * from in_app_dna_info_mapping where company_id is not null and publisher_id is null  limit 3;

--select count(company_id) from dna_universal_company_mapping where  confirmed = 'true';

-- select count(company_id) from dna_universal_company_mapping where  confirmed = 'true';

-- select * from dna_universal_company_office limit 3;
-- select * from in_app_dna_info_mapping limit 3;
--\d
-- select * from dna_universal_company_parent where comments like '%acquire%' limit 5;
-- select * from in_app_dna_info_mapping where product_id=310633997  limit 3;
\x
-- select * from dna_universal_company_parent where universal_id=1000200000017524 limit 3;



--select * from in_app_dna_info_mapping where product_id=417871100  limit 3;
\x

-- select * from dna_universal_company_parent  limit 3;

select * from dna_universal_company_parent limit 3;
select * from ( select count(*) as count_parent_id, universal_id from dna_universal_company_parent group by parent_id,  universal_id ) as prod where count_parent_id>1 limit 3



--select * from in_app_dna_info_mapping where product_id= 1398851503

EOF

In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;

select * from dna_universal_company_office limit 3;
EOF

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;
-- \d public.dna_universal_company_mapping;
select * from public.dna_universal_company_mapping limit 3;
EOF

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;
\d 
select * from dna_universal_company_mapping where company_id=300166374  limit 3;
select * from dna_universal_company where id=1000200000000251 limit 3;
select * from in_company_publisher_mapping where company_id=1000200000000251  limit 3
EOF

In [0]:


spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/market_code=apple-store/batch_id=1/part-applestore-1.c000.gz.parquet").select("publisher_code","publisher",).show()

In [0]:
%%sh
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/market_code=apple-store/batch_id=1/part-applestore-1.c000.gz.parquet --human
aws s3 ls  s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-05-08/platform=ios/part-00000-1644f820-bc51-4510-b897-d3b81bf8691f-c000.snappy.parquet




# aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-tech.store.app.v1/dimension/market_code=apple-store/batch_id=1/part-applestore-1.c000.gz.parquet


In [0]:


spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-05-08/").createOrReplaceTempView("publisher_test")
spark.sql("SELECT * FROM publisher_test").show()

In [0]:

spark.sql("SELECT  DISTINCT COUNT(category_id) as count_category_id, id, store_id FROM publisher_test GROUP BY id, store_id, feed, platform_id, vertical order by count_category_id desc   ").show()